# Start

In [1]:
exp_index = 0
experiments = {}

In [3]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice

from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict, cross_validate, RepeatedKFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import PowerTransformer, StandardScaler, OneHotEncoder, MinMaxScaler, RobustScaler, QuantileTransformer, Normalizer, MaxAbsScaler, FunctionTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SelectKBest, SequentialFeatureSelector, SelectFromModel, RFE, RFECV, f_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD, FastICA, FactorAnalysis, SparsePCA, NMF, LatentDirichletAllocation

from sklearn.cluster import KMeans, Birch
from sklearn.mixture import GaussianMixture

# import regressors
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor, HistGradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor, PassiveAggressiveRegressor, Perceptron, RidgeClassifier, LogisticRegression
from sklearn.linear_model import Lasso, ElasticNet, Lars, BayesianRidge, ARDRegression, OrthogonalMatchingPursuit, HuberRegressor, TheilSenRegressor, RANSACRegressor
from sklearn.linear_model import LassoLars, LassoLarsIC
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV, LarsCV, OrthogonalMatchingPursuitCV, LassoLarsCV, BayesianRidge, LinearRegression
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.kernel_ridge import KernelRidge

# pandas deactivate future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

SUBMIT = True
USE_ORIGINAL = True
SEED = 15
SAMPLE = 1
TARGET = 'Strength'
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
# cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=SEED)

train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')
orig = pd.read_csv('datasets/ConcreteStrengthData.csv')

for i, X in enumerate([train, test, orig]):
    if 'id' in X.columns:
        X.drop(['id'], axis=1, inplace=True)

# Define test set with original data
if not SUBMIT:
    train, test = train_test_split(train, test_size=0.2, random_state=SEED) 

# Add original data to training set
if USE_ORIGINAL:
    train = pd.concat([train, orig], axis=0)
    train.reset_index(inplace=True, drop=True)

# Sampling for faster training
if SAMPLE < 1:
    train = train.sample(frac=SAMPLE, random_state=SEED)

# set training data
X_train = train.copy()
y_train = X_train.pop(TARGET)
X_test = test.copy()

if not SUBMIT:
    y_test = X_test.pop(TARGET)
else:
    y_test = None

# print shapes
print(f'Train shape: {train.shape}, Test shape: {test.shape}, Original shape: {orig.shape}')

# Feature engineering
class FeatureEngineering(TransformerMixin, BaseEstimator):

    def __init__(self, level=0) -> None:
        self.level = level

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        """ Add new features to the dataset"""
        X = X.copy()

        # # Sum of features
        if self.level >= 2:
            X['cement_slag'] = X['CementComponent'] + X['BlastFurnaceSlag']
            X['cement_flyash'] = X['CementComponent'] + X['FlyAshComponent']
            X['cement_water'] = X['CementComponent'] + X['WaterComponent']
            X['cement_superplasticizer'] = X['CementComponent'] + X['SuperplasticizerComponent']
            X['cement_coarseaggregate'] = X['CementComponent'] + X['CoarseAggregateComponent']
            X['cement_fineaggregate'] = X['CementComponent'] + X['FineAggregateComponent']
            X['cement_age'] = X['CementComponent'] + X['AgeInDays']
            X['slag_flyash'] = X['BlastFurnaceSlag'] + X['FlyAshComponent']
            X['slag_water'] = X['BlastFurnaceSlag'] + X['WaterComponent']
            X['slag_superplasticizer'] = X['BlastFurnaceSlag'] + X['SuperplasticizerComponent']
            X['slag_coarseaggregate'] = X['BlastFurnaceSlag'] + X['CoarseAggregateComponent']
            X['slag_fineaggregate'] = X['BlastFurnaceSlag'] + X['FineAggregateComponent']
            X['slag_age'] = X['BlastFurnaceSlag'] + X['AgeInDays']
            X['flyash_water'] = X['FlyAshComponent'] + X['WaterComponent']
            X['flyash_superplasticizer'] = X['FlyAshComponent'] + X['SuperplasticizerComponent']
            X['flyash_coarseaggregate'] = X['FlyAshComponent'] + X['CoarseAggregateComponent']

        # # Ratio of features
        if self.level >= 2:
            # X['cement_slag_ratio'] = X['CementComponent'] / X['BlastFurnaceSlag']  # Inf values
            # X['cement_flyash_ratio'] = X['CementComponent'] / X['FlyAshComponent']  # Inf values
            X['cement_water_ratio'] = X['CementComponent'] / X['WaterComponent']
            # X['cement_superplasticizer_ratio'] = X['CementComponent'] / X['SuperplasticizerComponent']  # Inf values
            X['cement_coarseaggregate_ratio'] = X['CementComponent'] / X['CoarseAggregateComponent']
            X['cement_fineaggregate_ratio'] = X['CementComponent'] / X['FineAggregateComponent']
            X['cement_age_ratio'] = X['CementComponent'] / X['AgeInDays']
            # X['slag_flyash_ratio'] = X['BlastFurnaceSlag'] / X['FlyAshComponent'] # Divide by zero
            X['slag_water_ratio'] = X['BlastFurnaceSlag'] / X['WaterComponent']
            # X['slag_superplasticizer_ratio'] = X['BlastFurnaceSlag'] / X['SuperplasticizerComponent']  # Divide by zero
            X['slag_coarseaggregate_ratio'] = X['BlastFurnaceSlag'] / X['CoarseAggregateComponent']
            X['slag_fineaggregate_ratio'] = X['BlastFurnaceSlag'] / X['FineAggregateComponent']
            X['slag_age_ratio'] = X['BlastFurnaceSlag'] / X['AgeInDays']
            X['flyash_water_ratio'] = X['FlyAshComponent'] / X['WaterComponent']
            # X['flyash_superplasticizer_ratio'] = X['FlyAshComponent'] / X['SuperplasticizerComponent'] # Divide by zero
            X['flyash_coarseaggregate_ratio'] = X['FlyAshComponent'] / X['CoarseAggregateComponent']

        # Other features
        if self.level >= 1:
            X['water_age_cement_ratio'] = (X['WaterComponent'] * X['AgeInDays']) / X['CementComponent']
        if self.level >= 3:
            X['has_superplasticizer'] = X['SuperplasticizerComponent'].apply(lambda x: 1 if x > 0 else 0)
            X['has_flyash'] = X['FlyAshComponent'].apply(lambda x: 1 if x > 0 else 0)
            X['has_slag'] = X['BlastFurnaceSlag'].apply(lambda x: 1 if x > 0 else 0)

        # Taken from PHONG NGUYEN: Detailed feature description and feature engineering by ChatGPT
        if self.level >= 3:
            # Calculate TotalComponentWeight
            X['TotalComponentWeight'] = X['CementComponent'] + X['BlastFurnaceSlag'] + X['FlyAshComponent'] + X['WaterComponent'] + X['SuperplasticizerComponent'] + X['CoarseAggregateComponent'] + X['FineAggregateComponent']
            # Calculate Water-Cement-Ratio (WCR)
            X['WCR'] = X['WaterComponent'] / X['CementComponent']
            # Calculate Aggregate-Ratio (AR)
            X['AR'] = (X['CoarseAggregateComponent'] + X['FineAggregateComponent']) / X['CementComponent']
            # Calculate Cement-Age
            X['Cement-Age'] = X['CementComponent'] * X['AgeInDays']
        if self.level >= 2:
            # Calculate Water-Cement-Plus-Pozzolan-Ratio (WCPR)
            X['WCPR'] = X['WaterComponent'] / (X['CementComponent'] + X['BlastFurnaceSlag'] + X['FlyAshComponent'])
        
        self.feature_names_out_ = X.columns.tolist()
        return X
    
    def get_feature_names_out(self):
        return self.feature_names_out_
    

class ClusterFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=5, random_state=None, ohe=True):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.ohe = ohe
        self.cluster_models = {
            'KMeans': KMeans(n_clusters=n_clusters, random_state=random_state),
            'Birch': Birch(n_clusters=n_clusters),
            # 'MiniBatchKMeans': MiniBatchKMeans(n_clusters=n_clusters, random_state=random_state),
            'GaussianMixture': GaussianMixture(n_components=n_clusters, random_state=random_state),
        }
        self.feature_names_out_ = None

    def fit(self, X, y=None):
        for model in self.cluster_models.values():
            model.fit(X)
        return self
    
    def transform(self, X):
        X_out = X.copy()
        for model_name, model in self.cluster_models.items():
            X_out[f'{model_name}_{self.n_clusters}'] = model.predict(X)
        if self.ohe:
            X_out = pd.get_dummies(X_out, columns=[f'{model_name}_{self.n_clusters}' for model_name in self.cluster_models.keys()])
        self.feature_names_out_ = X_out.columns
        return X_out
    
    def get_feature_names_out(self):
        return self.feature_names_out_



Train shape: (6437, 9), Test shape: (3605, 8), Original shape: (1030, 9)


# Pipeline

In [32]:
def get_pipeline(regressor, model_name):
    pipeline = make_pipeline(
        # PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
        # FeatureEngineering(level=2),
        # ClusterFeatures(n_clusters=7, random_state=SEED, ohe=True),

        # StandardScaler(),
        PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
        # QuantileTransformer(output_distribution='normal', random_state=SEED),
        # RobustScaler(quantile_range=(5.0, 95.0), with_centering=True, with_scaling=True),
        # MinMaxScaler(feature_range=(0, 1)),

        # PCA(n_components=20, random_state=SEED),
        # KernelPCA(n_components=20, kernel='rbf', random_state=SEED),

        # SelectKBest(score_func=f_regression, k=30),
        # SelectFromModel(regressor, max_features=20),
        # SequentialFeatureSelector(regressor, n_features_to_select=20),

        regressor,
        # TransformedTargetRegressor(regressor=regressor, transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
    )
    return pipeline

# Cross validation

In [33]:
set_config(transform_output="pandas")  # set to pandas

# Set categorical features for catboost
cat_features = [col for col in X_train.columns if X_train[col].dtype == 'category']

regressors = {
    # 'LGBMRegressor1': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt'),
    # 'LGBMRegressor2': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='dart'),
    # 'LGBMRegressor3': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='goss'),
    # 'LGBMRegressor4': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='rf', subsample=.632, subsample_freq=1),
    # 'LGBMRegressor5': LGBMRegressor(random_state=SEED, n_jobs=-1, class_weight='balanced'),
    # 'LGBMRegressor6': LGBMRegressor(random_state=SEED, n_jobs=-1, subsample=0.7),
    # 'LGBMRegressor7': LGBMRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=0.7),
    # 'LGBMRegressor8': LGBMRegressor(random_state=SEED, n_jobs=-1, subsample=0.7, colsample_bytree=0.7),
    # 'LGBMRegressor9': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='dart', colsample_bytree=0.7),
    # 'LGBMRegressor10': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt', num_leaves=48, max_depth=14, learning_rate=0.08, n_estimators=240),
    # 'LGBMRegressor11': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt', num_leaves=48, max_depth=14, learning_rate=0.08, n_estimators=240, subsample=0.7, colsample_bytree=0.6),
    # 'LGBMRegressor12': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt', num_leaves=250, max_depth=3, learning_rate=0.015, n_estimators=600, subsample=0.65, colsample_bytree=0.9, reg_alpha=50),
    # 'XGBRegressor1': XGBRegressor(random_state=SEED, n_jobs=-1),
    # 'XGBRegressor2': XGBRegressor(random_state=SEED, n_jobs=-1, booster='dart'),
    # 'XGBRegressor3': XGBRegressor(random_state=SEED, n_jobs=-1, booster='gblinear'),
    # 'XGBRegressor4': XGBRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=0.7),
    # 'XGBRegressor5': XGBRegressor(random_state=SEED, n_jobs=-1, subsample=0.7),
    # 'XGBRegressor6': XGBRegressor(random_state=SEED, 
    #                               n_jobs=-1, 
    #                               learning_rate=0.055, 
    #                               n_estimators=200, 
    #                               max_depth=8, 
    #                               min_child_weight=1, 
    #                               gamma=0.07, 
    #                               colsample_bytree=0.67, 
    #                               colsample_bylevel=0.67, 
    #                               colsample_bynode=0.8,
    #                               subsample=0.7, 
    #                               objective='reg:squarederror'),
    # 'XGBRegressor7': XGBRegressor(random_state=SEED, n_jobs=-1, objective='reg:squarederror'),
    # 'XGBRandomForestRegressor': XGBRFRegressor(random_state=SEED, n_jobs=-1),
    # 'CatBoostRegressor': CatBoostRegressor(random_state=SEED, silent=True),
    # 'HistGradientBoostingRegressor': HistGradientBoostingRegressor(random_state=SEED),
    # 'HistGradientBoostingRegressor2': HistGradientBoostingRegressor(random_state=SEED, max_iter=200, 
    #                                                                 max_depth=6, learning_rate=0.1, 
    #                                                                 l2_regularization=0.0006754828207682132, max_leaf_nodes=42, 
    #                                                                 min_samples_leaf=9, max_bins=255),
    # 'HistGradientBoostingRegressor3': HistGradientBoostingRegressor(random_state=SEED, max_iter=1000, 
    #                                                                 max_depth=10, learning_rate=0.1, 
    #                                                                 l2_regularization=0.1, max_leaf_nodes=100, 
    #                                                                 min_samples_leaf=20, max_bins=255),
    # 'RandomForestRegressor': RandomForestRegressor(random_state=SEED, n_jobs=-1),
    # 'ExtraTreesRegressor': ExtraTreesRegressor(random_state=SEED, n_jobs=-1),
    # 'AdaBoostRegressor': AdaBoostRegressor(random_state=SEED),
    # 'GradientBoostingRegressor': GradientBoostingRegressor(random_state=SEED),
    # 'BaggingRegressor': BaggingRegressor(random_state=SEED, n_jobs=-1),
    # 'KNeighborsRegressor': KNeighborsRegressor(n_jobs=-1),
    # # 'DecisionTreeRegressor': DecisionTreeRegressor(random_state=SEED),
    # # 'GaussianProcessRegressor': GaussianProcessRegressor(random_state=SEED),
    'MLPRegressor1': MLPRegressor(random_state=SEED, max_iter=1000, activation='relu', solver='adam'),
    # # 'MLPRegressor2': MLPRegressor(random_state=SEED, max_iter=1000, activation='relu', solver='lbfgs'),
    # 'MLPRegressor3': MLPRegressor(random_state=SEED, max_iter=5000, activation='tanh', solver='adam'),
    # # 'MLPRegressor4': MLPRegressor(random_state=SEED, max_iter=1000, activation='tanh', solver='lbfgs'),
    # 'MLPRegressor5': MLPRegressor(random_state=SEED, max_iter=1000, activation='logistic', solver='adam'),
    # # 'MLPRegressor6': MLPRegressor(random_state=SEED, max_iter=1000, activation='logistic', solver='lbfgs'),
    # 'MLPRegressor7': MLPRegressor(random_state=SEED, max_iter=1000, activation='identity', solver='adam'),
    # 'MLPRegressor8': MLPRegressor(random_state=SEED, max_iter=1000, activation='identity', solver='lbfgs'),
    # # 'Ridge': Ridge(random_state=SEED),
    # 'RidgeCV': RidgeCV(alphas=np.logspace(-3, 3, 7), cv=cv),
    # 'KernelRidge': KernelRidge(kernel='polynomial', degree=2, coef0=2.5),
    # # 'SGDRegressor': SGDRegressor(random_state=SEED, max_iter=1000, tol=1e-3),
    # # 'PassiveAggressiveRegressor': PassiveAggressiveRegressor(random_state=SEED, max_iter=1000, tol=1e-3),
    # # 'LinearRegression': LinearRegression(n_jobs=-1),
    # # 'Lasso': Lasso(random_state=SEED),
    # # 'ElasticNet': ElasticNet(random_state=SEED, max_iter=1000000),
    # 'ElasticNetCV': ElasticNetCV(alphas=np.logspace(-3, 3, 7), cv=cv, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=1000000),
    # # 'HuberRegressor': HuberRegressor(max_iter=1000),
    # 'BayesianRidge': BayesianRidge(),
    # # 'ARDRegression': ARDRegression(),
    # # 'TheilSenRegressor': TheilSenRegressor(random_state=SEED),
    # # 'RANSACRegressor': RANSACRegressor(random_state=SEED),
    # # 'OrthogonalMatchingPursuit': OrthogonalMatchingPursuit(normalize=False),
    # # 'Lars': Lars(),
    # # 'LassoLars': LassoLars(),
    # # 'LassoLarsIC': LassoLarsIC(normalize=False),
    # 'SVR': SVR(),
    # 'NuSVR': NuSVR(),
    # 'LinearSVR': LinearSVR(max_iter=1000000),
    # #         )
}

SCALING_MODELS = []  # TODO remove when pipeline is fixed
exp_index += 1
print(f'Experiment {exp_index}: {len(regressors)} models')

for model_name, regressor in regressors.items():
    t0 = time.time()
    feature_importances = pd.DataFrame()

    pipeline = get_pipeline(regressor, model_name)
    
    scoring = {'rmse': make_scorer(mean_squared_error, squared=False)}
    results = cross_validate(pipeline, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1, return_estimator=True, verbose=0)
    scores = results['test_rmse']

    # # get feature importances
    # for i, estimator in enumerate(results['estimator']):
    #     regressor_ = estimator._final_estimator
    #     try:
    #         feature_importance = pd.Series(regressor_.feature_importances_, index=estimator[-2].get_feature_names_out(), name=f'fold{i}')
    #     except:
    #         feature_importance = pd.Series(regressor_.coef_, index=estimator[-2].get_feature_names_out(), name=f'fold{i}')
    #     feature_importances = pd.concat([feature_importances, feature_importance], axis=1)
    # feature_importances['mean'] = feature_importances.mean(axis=1)
    
    # fit on all train data and predict on test
    pipeline.fit(X_train, y_train)        
    y_pred = pipeline.predict(X_test)
    
    if not SUBMIT:
        score_eval = mean_squared_error(y_test, y_pred, squared=False)
    else:
        score_eval = np.nan
    
    print(f'{model_name}: {np.mean(scores):.4f} ± {np.std(scores):.4f}, Time: {time.time() - t0:.2f} seconds, RMSE: {score_eval:.4f}')
    # print(feature_importances.sort_values('mean', ascending=False))

    # Save experiment (index, model, scores, std, fit_time)
    experiments[f'{model_name}_exp{exp_index}'] = (model_name, exp_index, pipeline, np.mean(scores), np.std(scores), time.time() - t0, score_eval)
set_config(transform_output="default")  # reset to default

Experiment 8: 1 models
MLPRegressor1: 11.8310 ± 0.2090, Time: 22.54 seconds, RMSE: 12.2906


In [34]:
now = time.strftime("%Y-%m-%d %H_%M_%S")
experiments_df = pd.DataFrame.from_dict(experiments, orient='index', columns=['model', 'exp_index', 'pipeline', 'mean', 'std', 'fit_time', 'score_eval'])
experiments_df['mean+2std'] = experiments_df['mean'] + 2 * experiments_df['std']
experiments_df.to_excel(f'experiments/experiments_df_{now}.xlsx')
experiments_df.sort_values('mean+2std').head(30)

,model,exp_index,pipeline,mean,std,fit_time,score_eval,mean+2std
MLPRegressor1_exp1,MLPRegressor1,1,"(PowerTransformer(), MLPRegressor(max_iter=100...",11.635783,0.245801,14.151253,12.214778,12.127386
MLPRegressor1_exp4,MLPRegressor1,4,"(PolynomialFeatures(include_bias=False, intera...",11.657544,0.256553,9.555548,12.237993,12.170651
MLPRegressor1_exp7,MLPRegressor1,7,"(PowerTransformer(), TransformedTargetRegresso...",11.712246,0.241655,4.989760,12.395502,12.195555
MLPRegressor1_exp5,MLPRegressor1,5,"(PolynomialFeatures(include_bias=False, intera...",11.710103,0.244583,10.764873,12.353874,12.199269
MLPRegressor1_exp6,MLPRegressor1,6,"(StandardScaler(), MLPRegressor(max_iter=1000,...",11.693735,0.254951,9.437719,12.300603,12.203637
MLPRegressor1_exp3,MLPRegressor1,3,"(PolynomialFeatures(include_bias=False, intera...",11.642781,0.285346,10.177981,12.222367,12.213472
MLPRegressor1_exp2,MLPRegressor1,2,"(PolynomialFeatures(include_bias=False, intera...",11.660341,0.292407,10.267750,12.204506,12.245154
MLPRegressor1_exp8,MLPRegressor1,8,"(ClusterFeatures(n_clusters=7, random_state=15...",11.830999,0.208977,22.545705,12.290617,12.248954


# Optuna

## Optuna LGBMRegressor

In [46]:
def objective(trial):

    regressor = LGBMRegressor(
        random_state=SEED,
        n_jobs=-1,
        # boosting_type=trial.suggest_categorical('boosting_type', ['gbdt', 'rf']),
        num_leaves=trial.suggest_int('num_leaves', 2, 256),
        max_depth=trial.suggest_int('max_depth', 2, 16),
        learning_rate=trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        n_estimators=trial.suggest_int('n_estimators', 100, 1000),
        subsample=trial.suggest_uniform('subsample', 0.632, 1.0),
        subsample_freq=trial.suggest_int('subsample_freq', 1, 10),
        colsample_bytree=trial.suggest_uniform('colsample_bytree', 0.632, 1.0),
        reg_alpha=trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
    )

    scoring = {'rmse': make_scorer(mean_squared_error, squared=False)}
    results = cross_validate(regressor, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1, return_estimator=True, verbose=0)
    scores = results['test_rmse']
    print(f'RMSE: {np.mean(scores):.4f} ± {np.std(scores):.4f}')
    return np.mean(scores) + 2 * np.std(scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
df_study = study.trials_dataframe()
df_study.to_excel(f'optuna/study_LGBM.xlsx')

print(f'Best trial: {study.best_trial.value:.4f}')
print(f'Best params: {study.best_params}')


[I 2023-03-09 12:00:10,709] A new study created in memory with name: no-name-f782f036-3ab1-44a9-8c1c-4b23178abf8b
[I 2023-03-09 12:00:15,365] Trial 0 finished with value: 12.173066089854595 and parameters: {'num_leaves': 176, 'max_depth': 4, 'learning_rate': 0.01739575994396942, 'n_estimators': 915, 'subsample': 0.6957462980264707, 'subsample_freq': 5, 'colsample_bytree': 0.6855648743058261, 'reg_alpha': 0.011937893553968165}. Best is trial 0 with value: 12.173066089854595.


RMSE: 11.6141 ± 0.2795


[I 2023-03-09 12:00:19,048] Trial 1 finished with value: 12.657859152698926 and parameters: {'num_leaves': 209, 'max_depth': 10, 'learning_rate': 0.002509457009565883, 'n_estimators': 663, 'subsample': 0.6753292821852301, 'subsample_freq': 1, 'colsample_bytree': 0.6819047027470442, 'reg_alpha': 1.2398175989388711}. Best is trial 0 with value: 12.173066089854595.


RMSE: 12.1270 ± 0.2654


[I 2023-03-09 12:00:21,451] Trial 2 finished with value: 12.73580778951298 and parameters: {'num_leaves': 81, 'max_depth': 13, 'learning_rate': 0.0038996292509704926, 'n_estimators': 337, 'subsample': 0.6520050181866903, 'subsample_freq': 5, 'colsample_bytree': 0.7613756636078657, 'reg_alpha': 0.04108891414826034}. Best is trial 0 with value: 12.173066089854595.


...

[I 2023-03-09 12:01:13,896] Trial 97 finished with value: 12.075383850890434 and parameters: {'num_leaves': 131, 'max_depth': 3, 'learning_rate': 0.017488772062603134, 'n_estimators': 335, 'subsample': 0.6781175696763901, 'subsample_freq': 4, 'colsample_bytree': 0.9589019518519364, 'reg_alpha': 0.015061044311099682}. Best is trial 74 with value: 12.024534365292915.


RMSE: 11.5590 ± 0.2582


[I 2023-03-09 12:01:14,743] Trial 98 finished with value: 12.552052780876009 and parameters: {'num_leaves': 221, 'max_depth': 11, 'learning_rate': 0.04065281221637223, 'n_estimators': 263, 'subsample': 0.773772151603933, 'subsample_freq': 5, 'colsample_bytree': 0.9452794859821366, 'reg_alpha': 0.14861161276340362}. Best is trial 74 with value: 12.024534365292915.
[I 2023-03-09 12:01:14,845] Trial 99 finished with value: 12.602972954466923 and parameters: {'num_leaves': 169, 'max_depth': 2, 'learning_rate': 0.01975011018980145, 'n_estimators': 117, 'subsample': 0.808344568960078, 'subsample_freq': 10, 'colsample_bytree': 0.9775154275933822, 'reg_alpha': 0.007109397001095117}. Best is trial 74 with value: 12.024534365292915.


RMSE: 12.0477 ± 0.2522
RMSE: 12.2141 ± 0.1944


PermissionError: [Errno 13] Permission denied: 'optuna/study_LGBM.xlsx'

In [47]:
df_study.to_excel(f'optuna/study_LGBM.xlsx')

print(f'Best trial: {study.best_trial.value:.4f}')
print(f'Best params: {study.best_params}')

Best trial: 12.0245
Best params: {'num_leaves': 161, 'max_depth': 2, 'learning_rate': 0.03189823639272201, 'n_estimators': 277, 'subsample': 0.7623316885643929, 'subsample_freq': 8, 'colsample_bytree': 0.6539749148962364, 'reg_alpha': 0.10648649504538502}


In [60]:
plot_optimization_history(study)

In [61]:
plot_param_importances(study)

In [62]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

In [63]:
plot_slice(study)

## Optuna XGBRegressor

In [48]:
def objective(trial):

    regressor = XGBRegressor(
        random_state=SEED,
        n_jobs=-1,
        booster=trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        max_depth=trial.suggest_int('max_depth', 2, 16),
        learning_rate=trial.suggest_loguniform('learning_rate', 1e-2, 1e-1),
        n_estimators=trial.suggest_int('n_estimators', 100, 1000),
        subsample=trial.suggest_uniform('subsample', 0.632, 1.0),
        colsample_bytree=trial.suggest_uniform('colsample_bytree', 0.632, 1.0),
        reg_alpha=trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
        reg_lambda=trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
        eval_metric = 'rmse',
        objective = 'reg:squarederror',
    )

    scoring = {'rmse': make_scorer(mean_squared_error, squared=False)}
    results = cross_validate(regressor, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1, return_estimator=True, verbose=0)
    scores = results['test_rmse']
    print(f'RMSE: {np.mean(scores):.4f} ± {np.std(scores):.4f}')
    return np.mean(scores) + 2 * np.std(scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
df_study = study.trials_dataframe()
df_study.to_excel(f'optuna/study_XGBRegressor.xlsx')
print(f'Best trial: {study.best_trial.value:.4f}')
print(f'Best params: {study.best_params}')


[I 2023-03-09 12:05:02,848] A new study created in memory with name: no-name-182fb4e0-37c1-424e-aaf2-b3c731f31db4
[I 2023-03-09 12:05:13,686] Trial 0 finished with value: 12.186291724447672 and parameters: {'booster': 'dart', 'max_depth': 3, 'learning_rate': 0.032918686300455045, 'n_estimators': 352, 'subsample': 0.6673418225380843, 'colsample_bytree': 0.7526486117871103, 'reg_alpha': 2.720541482014314, 'reg_lambda': 722.2718189485228}. Best is trial 0 with value: 12.186291724447672.


RMSE: 11.6459 ± 0.2702


[I 2023-03-09 12:05:13,939] Trial 1 finished with value: 17.017592576321857 and parameters: {'booster': 'gblinear', 'max_depth': 15, 'learning_rate': 0.04937208370050335, 'n_estimators': 455, 'subsample': 0.9657947778172493, 'colsample_bytree': 0.9905150520293695, 'reg_alpha': 684.4237939758232, 'reg_lambda': 0.0545071718651421}. Best is trial 0 with value: 12.186291724447672.


RMSE: 16.5847 ± 0.2165


[I 2023-03-09 12:05:14,382] Trial 2 finished with value: 15.374733828835009 and parameters: {'booster': 'gblinear', 'max_depth': 6, 'learning_rate': 0.06433674427738928, 'n_estimators': 989, 'subsample': 0.7758504720801369, 'colsample_bytree': 0.9237769007558556, 'reg_alpha': 90.3823102990961, 'reg_lambda': 17.91866757239939}. Best is trial 0 with value: 12.186291724447672.


...

RMSE: 14.3187 ± 0.2772


[I 2023-03-09 13:04:18,281] Trial 98 finished with value: 12.10668212756606 and parameters: {'booster': 'dart', 'max_depth': 3, 'learning_rate': 0.013774760217326787, 'n_estimators': 890, 'subsample': 0.7961260680557372, 'colsample_bytree': 0.7289965474500437, 'reg_alpha': 0.015408648730059354, 'reg_lambda': 1.331858635308501}. Best is trial 93 with value: 12.050694123212697.


RMSE: 11.5608 ± 0.2729


[I 2023-03-09 13:05:33,043] Trial 99 finished with value: 12.175712930844202 and parameters: {'booster': 'dart', 'max_depth': 4, 'learning_rate': 0.014367753239656236, 'n_estimators': 941, 'subsample': 0.788304695019252, 'colsample_bytree': 0.6861057023308899, 'reg_alpha': 0.0015983269261056633, 'reg_lambda': 3.1879254487057525}. Best is trial 93 with value: 12.050694123212697.


RMSE: 11.6131 ± 0.2813
Best trial: 12.0507
Best params: {'booster': 'dart', 'max_depth': 2, 'learning_rate': 0.012642475908088945, 'n_estimators': 855, 'subsample': 0.8121633566805094, 'colsample_bytree': 0.7117066890547505, 'reg_alpha': 0.005539220298259456, 'reg_lambda': 0.7699973173912055}


## Optuna MLPRegressor

In [45]:
def objective(trial):
    t0 = time.time()
    hidden_layer_1 = trial.suggest_categorical("hidden_layer_1", [50, 100, 200, 500])
    hidden_layer_2 = trial.suggest_categorical("hidden_layer_2", [0, 50, 100, 200, 500])
    # hidden_layer_3 = trial.suggest_int("hidden_layer_3", 0, 500)
    hidden_layer_3 = 0

    if hidden_layer_2 == 0:
        hidden_layer_sizes = (hidden_layer_1,)
    elif hidden_layer_3 == 0:
        hidden_layer_sizes = (hidden_layer_1, hidden_layer_2)
    else:
        hidden_layer_sizes = (hidden_layer_1, hidden_layer_2, hidden_layer_3)
        
    regressor = MLPRegressor(
        # activation=trial.suggest_categorical("activation", ['identity', 'logistic', 'tanh', 'relu']),
        activation=trial.suggest_categorical("activation", ['logistic', 'relu']),
        solver=trial.suggest_categorical("solver", ['sgd', 'adam']),
        # learning_rate=trial.suggest_categorical("learning_rate", ['constant', 'invscaling', 'adaptive']),
        hidden_layer_sizes=hidden_layer_sizes,
        # alpha=trial.suggest_loguniform("alpha", 1e-5, 1e-1),
        max_iter=1000,
        random_state=SEED,
    )

    pipeline = make_pipeline(
        PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
        regressor
    )

    scoring = {'rmse': make_scorer(mean_squared_error, squared=False)}
    results = cross_validate(pipeline, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1, return_estimator=True, verbose=0)
    scores = results['test_rmse']
    print(f'Score: {np.mean(scores):.4f}, std: {np.std(scores):.4f}, fit time: {time.time() - t0:.2f}s')
    return np.mean(scores) + 2 * np.std(scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
df_study = study.trials_dataframe()
df_study.to_excel(f'optuna/study_MLPRegressor.xlsx')
print(f'Best trial: {study.best_trial.value:.4f}')
print(f'Best params: {study.best_params}')

[I 2023-03-09 11:22:59,355] A new study created in memory with name: no-name-d203d976-5f7a-4f4d-be30-0e2eb076942b
[I 2023-03-09 11:23:12,163] Trial 0 finished with value: 12.51285591080878 and parameters: {'hidden_layer_1': 50, 'hidden_layer_2': 100, 'activation': 'relu', 'solver': 'adam'}. Best is trial 0 with value: 12.51285591080878.


Score: 11.8422, std: 0.3353, fit time: 12.80s


[I 2023-03-09 11:23:28,390] Trial 1 finished with value: 12.191331198349221 and parameters: {'hidden_layer_1': 500, 'hidden_layer_2': 0, 'activation': 'relu', 'solver': 'adam'}. Best is trial 1 with value: 12.191331198349221.


Score: 11.6527, std: 0.2693, fit time: 16.23s


[I 2023-03-09 11:23:48,072] Trial 2 finished with value: 12.141521870854735 and parameters: {'hidden_layer_1': 200, 'hidden_layer_2': 0, 'activation': 'logistic', 'solver': 'adam'}. Best is trial 2 with value: 12.141521870854735.


...

Score: 11.6562, std: 0.2717, fit time: 14.71s


[I 2023-03-09 11:28:34,615] Trial 18 finished with value: 12.135993589762375 and parameters: {'hidden_layer_1': 100, 'hidden_layer_2': 0, 'activation': 'logistic', 'solver': 'adam'}. Best is trial 15 with value: 12.135993589762375.


Score: 11.6265, std: 0.2547, fit time: 11.40s


[I 2023-03-09 11:28:45,929] Trial 19 finished with value: 12.135993589762375 and parameters: {'hidden_layer_1': 100, 'hidden_layer_2': 0, 'activation': 'logistic', 'solver': 'adam'}. Best is trial 15 with value: 12.135993589762375.


Score: 11.6265, std: 0.2547, fit time: 11.31s
Best trial: 12.1360
Best params: {'hidden_layer_1': 100, 'hidden_layer_2': 0, 'activation': 'logistic', 'solver': 'adam'}


In [ ]:
# Best trial: 11.6238 (mean score)
# Best params: {'hidden_layer_1': 225, 'hidden_layer_2': 182, 'hidden_layer_3': 132, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate': 'adaptive', 'alpha': 0.0008417896474524812}


In [41]:
plot_optimization_history(study)

In [42]:
plot_param_importances(study)

In [43]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

In [44]:
plot_slice(study)

## Optuna CatBoostRegressor

In [49]:
def objective(trial):

    regressor = CatBoostRegressor(
        learning_rate=trial.suggest_loguniform('learning_rate', 1e-2, 1e-1),
        n_estimators=trial.suggest_int('n_estimators', 100, 500),
        max_depth=trial.suggest_int('max_depth', 2, 12),
        l2_leaf_reg=trial.suggest_loguniform('l2_leaf_reg', 1e-3, 1e3),
        rsm=trial.suggest_uniform('rsm', 0.632, 1.0),
        bagging_temperature=trial.suggest_uniform('bagging_temperature', 0.632, 1.0),
        random_state=SEED,
        verbose=0,
        early_stopping_rounds=100,
        eval_metric='RMSE',
    )

    scoring = {'rmse': make_scorer(mean_squared_error, squared=False)}
    results = cross_validate(regressor, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1, return_estimator=True, verbose=0)
    scores = results['test_rmse']
    print(f'RMSE: {np.mean(scores):.4f} ± {np.std(scores):.4f}')
    return np.mean(scores) + 2 * np.std(scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
df_study = study.trials_dataframe()
df_study.to_excel(f'optuna/study_CatBoostRegressor.xlsx')

[I 2023-03-09 13:34:45,008] A new study created in memory with name: no-name-598a6617-4ce8-4837-b75c-687b6b05a12c
[I 2023-03-09 13:34:59,518] Trial 0 finished with value: 12.090284015956973 and parameters: {'learning_rate': 0.019297144863715514, 'n_estimators': 227, 'max_depth': 11, 'l2_leaf_reg': 8.86551238356123, 'rsm': 0.6906289444564622, 'bagging_temperature': 0.896746729603694}. Best is trial 0 with value: 12.090284015956973.


RMSE: 11.6276 ± 0.2314


[I 2023-03-09 13:35:01,826] Trial 1 finished with value: 12.045153399024315 and parameters: {'learning_rate': 0.06948695478341943, 'n_estimators': 155, 'max_depth': 5, 'l2_leaf_reg': 0.7353756372999178, 'rsm': 0.8612186042943057, 'bagging_temperature': 0.9655530822914353}. Best is trial 1 with value: 12.045153399024315.


RMSE: 11.5409 ± 0.2521


[I 2023-03-09 13:35:10,224] Trial 2 finished with value: 12.13310760410373 and parameters: {'learning_rate': 0.018999578774639057, 'n_estimators': 448, 'max_depth': 9, 'l2_leaf_reg': 0.9730494881815308, 'rsm': 0.7962288062695819, 'bagging_temperature': 0.7301814688416073}. Best is trial 1 with value: 12.045153399024315.


...

RMSE: 11.5764 ± 0.2515


[I 2023-03-09 13:36:14,842] Trial 48 finished with value: 12.054797306663742 and parameters: {'learning_rate': 0.03165620327742269, 'n_estimators': 327, 'max_depth': 6, 'l2_leaf_reg': 1.725983635621939, 'rsm': 0.7832951060143298, 'bagging_temperature': 0.821628679823392}. Best is trial 33 with value: 12.021834752415936.


RMSE: 11.5475 ± 0.2537


[I 2023-03-09 13:36:17,737] Trial 49 finished with value: 12.077962529612202 and parameters: {'learning_rate': 0.02638340152902011, 'n_estimators': 267, 'max_depth': 8, 'l2_leaf_reg': 0.702823332271372, 'rsm': 0.8323106464436826, 'bagging_temperature': 0.7974503387958407}. Best is trial 33 with value: 12.021834752415936.


RMSE: 11.5665 ± 0.2557


In [50]:
print(f'Best trial: {study.best_trial.value:.4f}')
print(f'Best params: {study.best_params}')

Best trial: 12.0218
Best params: {'learning_rate': 0.037467902179125184, 'n_estimators': 205, 'max_depth': 5, 'l2_leaf_reg': 4.357005411240157, 'rsm': 0.7362139736946606, 'bagging_temperature': 0.8648719006111018}


## Optuna NuSVR

In [3]:
set_config(transform_output="pandas")  # set to pandas
def objective(trial):

    regressor = NuSVR(
        nu=trial.suggest_uniform('nu', 0.0, 1.0),
        kernel=trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
        degree=trial.suggest_int('degree', 2, 5),
        gamma=trial.suggest_categorical('gamma', ['scale', 'auto']),
        coef0=trial.suggest_uniform('coef0', 0.0, 1.0),
        tol=trial.suggest_loguniform('tol', 1e-5, 1e-1),
        C=trial.suggest_loguniform('C', 1e-3, 1e3),
        cache_size=trial.suggest_int('cache_size', 100, 1000),
        verbose=0,
    )

    scoring = {'rmse': make_scorer(mean_squared_error, squared=False)}
    pipeline = make_pipeline(
        PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
        FeatureEngineering(level=1),
        PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
        TransformedTargetRegressor(regressor=regressor, transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
    )
    results = cross_validate(pipeline, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1, return_estimator=True, verbose=0)
    scores = results['test_rmse']
    return np.mean(scores) + 2 * np.std(scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
df_study = study.trials_dataframe()
df_study.to_excel(f'optuna/study_NuSVR.xlsx')
set_config(transform_output="default")  # set to pandas

[I 2023-03-08 10:46:03,031] A new study created in memory with name: no-name-71645870-5140-4c64-bca3-0c4ec94156bb
[I 2023-03-08 10:46:07,227] Trial 0 finished with value: 12.21242940700217 and parameters: {'nu': 0.2521854911601381, 'kernel': 'poly', 'degree': 4, 'gamma': 'scale', 'coef0': 0.700347099837816, 'tol': 0.01982289294311887, 'C': 0.03509167134827058, 'cache_size': 639}. Best is trial 0 with value: 12.21242940700217.
[I 2023-03-08 10:46:11,251] Trial 1 finished with value: 12.292010273410968 and parameters: {'nu': 0.9713531132717544, 'kernel': 'rbf', 'degree': 2, 'gamma': 'scale', 'coef0': 0.8088871396726504, 'tol': 0.000288991271449246, 'C': 0.19006215124021775, 'cache_size': 973}. Best is trial 0 with value: 12.21242940700217.
[I 2023-03-08 10:46:13,211] Trial 2 finished with value: 31.29031475851029 and parameters: {'nu': 0.18860288854959106, 'kernel': 'sigmoid', 'degree': 4, 'gamma': 'scale', 'coef0': 0.2520538437869918, 'tol': 0.0005719315380673731, 'C': 0.097528784755083

In [7]:
plot_optimization_history(study)

In [8]:
plot_param_importances(study)

In [9]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

In [10]:
plot_slice(study)

## Optuna KernelRidge

In [10]:
set_config(transform_output="pandas")  # set to pandas
def objective(trial):

    regressor = KernelRidge(
        alpha=trial.suggest_loguniform('alpha', 1e-2, 1.0),
        kernel=trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf']),  # 'sigmoid' too slow and not promising
        degree=trial.suggest_int('degree', 2, 5),
        gamma=trial.suggest_float('gamma', 1e-3, 1e3, log=True),
        coef0=trial.suggest_float('coef0', 0.5, 1.0),
    )
    # print(regressor.get_params())
    scoring = {'rmse': make_scorer(mean_squared_error, squared=False)}
    pipeline = make_pipeline(
        PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
        FeatureEngineering(level=2),
        PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
        TransformedTargetRegressor(regressor=regressor, transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
    )
    results = cross_validate(pipeline, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False, n_jobs=-1, return_estimator=True, verbose=0)
    scores = results['test_rmse']
    return np.mean(scores) + 2 * np.std(scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
df_study = study.trials_dataframe()
df_study.to_excel(f'optuna/study_KernelRidge.xlsx')

print(f'Best trial: {study.best_trial.value:.4f}')
print(f'Best params: {study.best_params}')

set_config(transform_output="default")  # set to pandas

[I 2023-03-08 11:12:49,842] A new study created in memory with name: no-name-53e2cb6c-9749-487a-956a-154ffccfbc59
[I 2023-03-08 11:12:53,459] Trial 0 finished with value: 14.204980876513043 and parameters: {'alpha': 0.09266105799325869, 'kernel': 'rbf', 'degree': 3, 'gamma': 67.3933202867308, 'coef0': 0.9558138855471328}. Best is trial 0 with value: 14.204980876513043.
[I 2023-03-08 11:12:55,475] Trial 1 finished with value: 12.109020101711456 and parameters: {'alpha': 0.012224236037528065, 'kernel': 'linear', 'degree': 2, 'gamma': 0.11238931722504199, 'coef0': 0.8370425092051945}. Best is trial 1 with value: 12.109020101711456.
[I 2023-03-08 11:12:58,414] Trial 2 finished with value: 12.129729309702347 and parameters: {'alpha': 0.14885947505578126, 'kernel': 'poly', 'degree': 3, 'gamma': 0.001738965473334055, 'coef0': 0.7287650201803793}. Best is trial 1 with value: 12.109020101711456.
[I 2023-03-08 11:13:00,600] Trial 3 finished with value: 12.12664231139537 and parameters: {'alpha':

Best trial: 12.0613
Best params: {'alpha': 0.25280921801434975, 'kernel': 'rbf', 'degree': 5, 'gamma': 0.003945115559099816, 'coef0': 0.5104182738708964}


In [11]:
plot_optimization_history(study)

In [12]:
plot_param_importances(study)

In [13]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

In [14]:
plot_slice(study)

# Stacking

In [11]:
study.best_params

{'nu': 0.15217480245377052,
 'kernel': 'rbf',
 'degree': 4,
 'gamma': 'auto',
 'coef0': 0.8513742003151807,
 'tol': 3.2150198709986036e-05,
 'C': 1.3771250686128407,
 'cache_size': 690}

In [65]:
from mlxtend.regressor import StackingCVRegressor
set_config(transform_output="pandas")  # set to pandas
lgbm_params = {'num_leaves': 161, 'max_depth': 2, 'learning_rate': 0.03189823639272201, 'n_estimators': 277, 'subsample': 0.7623316885643929, 'subsample_freq': 8, 
               'colsample_bytree': 0.6539749148962364, 'reg_alpha': 0.10648649504538502, 'random_state': SEED}
xgb_params = {'booster': 'dart', 'max_depth': 2, 'learning_rate': 0.012642475908088945, 'n_estimators': 855, 'subsample': 0.8121633566805094, 
              'colsample_bytree': 0.7117066890547505, 'reg_alpha': 0.005539220298259456, 'reg_lambda': 0.7699973173912055, 'random_state': SEED}
catboost_params = {'learning_rate': 0.037467902179125184, 'n_estimators': 205, 'max_depth': 5, 'l2_leaf_reg': 4.357005411240157, 'rsm': 0.7362139736946606, 
                   'bagging_temperature': 0.8648719006111018, 'silent': True, 'random_state': SEED, 'eval_metric': 'RMSE'}
stack_model = StackingCVRegressor(
    regressors=(
        make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=1),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=NuSVR(nu=0.15,
                                                       kernel='rbf',
                                                       tol=0.001), 
                                                       transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
        ),
        make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=2),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=KernelRidge(alpha=0.25, kernel='rbf', degree=5, gamma=0.004, coef0=0.5),
                                       transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
        ),
        make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=1),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=MLPRegressor(activation='logistic', max_iter=1000,solver='adam', tol=0.0001)
                                       )
        ),
        LGBMRegressor(**lgbm_params),
        XGBRegressor(**xgb_params),
        CatBoostRegressor(**catboost_params),
    ),
    meta_regressor=RidgeCV(alphas=[0.1, 1.0, 10.0], cv=cv, scoring='neg_root_mean_squared_error'),
    cv=cv,
    use_features_in_secondary=False,
    store_train_meta_features=False,
    shuffle=False,
    random_state=SEED,
    verbose=0,
    n_jobs=-1,
)

stack_model = StackingRegressor(
    estimators=(
        make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=1),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=NuSVR(nu=0.15,
                                                       kernel='rbf',
                                                       tol=0.001), 
                                                       transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
        ),
        make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=2),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=KernelRidge(alpha=0.25, kernel='rbf', degree=5, gamma=0.004, coef0=0.5),
                                       transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
        ),
        make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=1),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=MLPRegressor(activation='logistic', max_iter=1000,solver='adam', tol=0.0001)
                                       )
        ),
        LGBMRegressor(**lgbm_params),
        XGBRegressor(**xgb_params),
        CatBoostRegressor(**catboost_params),
    ),
    final_estimator=RidgeCV(alphas=np.logspace(-3, 3, 7), cv=cv, scoring='neg_root_mean_squared_error'),
    cv=cv,
    n_jobs=1,
)
scores = cross_val_score(stack_model, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=1, verbose=2)
print(f'CV RMSE: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

stack_model.fit(X_train, y_train)
y_pred = stack_model.predict(X_test)

if not SUBMIT:
    print(f'RMSE Test: {(mean_squared_error(y_test, y_pred, squared=False)):.4f}')
    
set_config(transform_output="default")

[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\hzf04d\Anaconda3\envs\EDA\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\hzf04d\Anaconda3\envs\EDA\lib\site-packages\sklearn\ensemble\_stacking.py", line 958, in fit
    return super().fit(X, y, sample_weight)
  File "c:\Users\hzf04d\Anaconda3\envs\EDA\lib\site-packages\sklearn\ensemble\_stacking.py", line 190, in fit
    self._validate_params()
  File "c:\Users\hzf04d\Anaconda3\envs\EDA\lib\site-packages\sklearn\base.py", line 581, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\hzf04d\Anaconda

In [67]:
# CV stacking: 
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

set_config(transform_output="pandas")  # set to pandas
lgbm_params = {'num_leaves': 161, 'max_depth': 2, 'learning_rate': 0.03189823639272201, 'n_estimators': 277, 'subsample': 0.7623316885643929, 'subsample_freq': 8, 
               'colsample_bytree': 0.6539749148962364, 'reg_alpha': 0.10648649504538502, 'random_state': SEED}
xgb_params = {'booster': 'dart', 'max_depth': 2, 'learning_rate': 0.012642475908088945, 'n_estimators': 855, 'subsample': 0.8121633566805094, 
              'colsample_bytree': 0.7117066890547505, 'reg_alpha': 0.005539220298259456, 'reg_lambda': 0.7699973173912055, 'random_state': SEED}
catboost_params = {'learning_rate': 0.037467902179125184, 'n_estimators': 205, 'max_depth': 5, 'l2_leaf_reg': 4.357005411240157, 'rsm': 0.7362139736946606, 
                   'bagging_temperature': 0.8648719006111018, 'silent': True, 'random_state': SEED, 'eval_metric': 'RMSE'}

regressors = {'NuSVR': make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
    FeatureEngineering(level=1),
    PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
    TransformedTargetRegressor(regressor=NuSVR(nu=0.15,
                                               kernel='rbf',
                                               tol=0.001), 
                                               transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
        ),
    'KernelRidge': make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=2),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=KernelRidge(alpha=0.25, kernel='rbf', degree=5, gamma=0.004, coef0=0.5),
                                       transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
        ),
    'MLPRegressor': make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=1),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=MLPRegressor(activation='logistic', max_iter=1000,solver='adam', tol=0.0001)
                                       )
        ),
    'LGBMRegressor': LGBMRegressor(**lgbm_params),
    'XGBRegressor': XGBRegressor(**xgb_params),
    'CatBoostRegressor': CatBoostRegressor(**catboost_params),
}

meta_regressors = [
    ('LinearRegression', LinearRegression()),
    ('RidgeCV', RidgeCV(alphas=np.logspace(-3, 3, 13), cv=cv)),
    ('ElasticNetCV', ElasticNetCV(alphas=np.logspace(-3, 3, 13), cv=cv, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=100000)),
    # ('LassoCV', LassoCV(alphas=np.logspace(-3, 3, 13), cv=cv, max_iter=100000)),  # = ElasticNetCV with l1_ratio=1
    # ('LarsCV', LarsCV(cv=cv, max_iter=100000, n_jobs=-1)),
    # ('OrthogonalMatchingPursuitCV', OrthogonalMatchingPursuitCV(cv=cv, n_jobs=-1)),
    # ('LassoLarsCV', LassoLarsCV(cv=cv, max_iter=10000, n_jobs=-1)),
    # ('BayesianRidge', BayesianRidge()),
]

FIT_REGRESSORS = True
DISPLAY_REGRESSOR_RESULTS = True
PLOT_ERRORS = False
TARGET_TRANSFORMATION = False

if FIT_REGRESSORS:
    # Store out of fold predictions for meta learner
    X_meta_trains = {}  # Dict of datasets used for meta learner training
    X_meta_hold_outs = {}  # Dict of datasets used for meta learner validation
    X_meta_tests = {}  # Dict of datasets used for meta learner testing

for i, (train_index, hold_out_index) in enumerate(cv.split(X_train)):
    t0 = time.time()
    print(f'Fold {i+1} of {cv.get_n_splits()}')
    X_train_cv, X_hold_out = X_train.iloc[train_index].copy(), X_train.iloc[hold_out_index].copy()
    y_train_cv, y_hold_out = y_train.iloc[train_index].copy(), y_train.iloc[hold_out_index].copy()
    
    X_meta_train = pd.DataFrame(index=train_index, columns=[name for name, _ in regressors.items()])
    X_meta_hold_out = pd.DataFrame(index=hold_out_index, columns=[name for name, _ in regressors.items()])
    X_meta_test = pd.DataFrame(index=X_test.index, columns=[name for name, _ in regressors.items()])
    
    if FIT_REGRESSORS:
        # for name, regressor in regressors:
        for name, regressor in regressors.items():
            print(f'Fitting {name} ...')
            if TARGET_TRANSFORMATION:
                ttr = TransformedTargetRegressor(regressor=regressor, func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)
            else:
                ttr = regressor
            
            if name == 'CatBoostRegressor':
                X_meta_train[name] = cross_val_predict(ttr, X_train_cv, y_train_cv, cv=cv, verbose=0)  # CatBoostRegressor fails on n_jobs=-1
            else:
                X_meta_train[name] = cross_val_predict(ttr, X_train_cv, y_train_cv, cv=cv, n_jobs=-1, verbose=0)
            
            # fit the model on the full cv training set
            ttr.fit(X_train_cv, y_train_cv)
            X_meta_hold_out[name] = ttr.predict(X_hold_out)
            X_meta_test[name] = ttr.predict(X_test)

            if DISPLAY_REGRESSOR_RESULTS:
                print(f'Hold out score of {name}: {mean_squared_error(y_hold_out, X_meta_hold_out[name], squared=False):.4f}')
                if not SUBMIT:
                    print(f'Test score of {name}: {mean_squared_error(y_test, X_meta_test[name], squared=False):.4f}')

        # Store datasets for meta learner
        X_meta_trains[i] = X_meta_train.copy()
        X_meta_hold_outs[i] = X_meta_hold_out.copy()
        X_meta_tests[i] = X_meta_test.copy()
        
    # Transform the predictions of regressors with target transform
    if TARGET_TRANSFORMATION:  
        X_meta_train = target_transform(X_meta_trains[i])
        X_meta_hold_out = target_transform(X_meta_hold_outs[i])
        X_meta_test = target_transform(X_meta_tests[i])
    else:
        X_meta_train = X_meta_trains[i]
        X_meta_hold_out = X_meta_hold_outs[i]
        X_meta_test = X_meta_tests[i]

    for name, meta_regressor in meta_regressors:
        # Fit the final estimator on the hold out predictions
        if TARGET_TRANSFORMATION:
            meta_ttr = TransformedTargetRegressor(
                meta_regressor,
                func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)
        else:
            meta_ttr = meta_regressor
        
        meta_ttr.fit(X_meta_train, y_train_cv)
        y_hold_out_pred = meta_ttr.predict(X_meta_hold_out)
        y_test_pred = meta_ttr.predict(X_meta_test)
        
        if not SUBMIT:
            score_eval = mean_squared_error(y_test, y_test_pred, squared=False)
        else:
            score_eval = np.nan
        
        l1_ratio = getattr(meta_ttr.regressor_, 'l1_ratio_', np.nan) if TARGET_TRANSFORMATION else getattr(meta_ttr, 'l1_ratio_', np.nan)
        alpha = getattr(meta_ttr.regressor_, 'alpha_', np.nan) if TARGET_TRANSFORMATION else getattr(meta_ttr, 'alpha_', np.nan)
        coef = getattr(meta_ttr.regressor_, 'coef_', np.nan) if TARGET_TRANSFORMATION else getattr(meta_ttr, 'coef_', np.nan)
        intercept = getattr(meta_ttr.regressor_, 'intercept_', np.nan) if TARGET_TRANSFORMATION else getattr(meta_ttr, 'intercept_', np.nan)
        
        print(f'Meta regressor: {name}, RMSE hold out: {mean_squared_error(y_hold_out, y_hold_out_pred, squared=False)},',
              f'RMSE test: {score_eval:.4f}, fit time: {time.time() - t0:.2f} s,',
              f'Coefficients: {coef}, intercept: {intercept}, l1_ratio: {l1_ratio}, alpha: {alpha}', end='\n'
        )
        if PLOT_ERRORS:
            plot_predictions_errors(y_hold_out, y_hold_out_pred, 'Hold out')
            if not SUBMIT:
                plot_predictions_errors(y_test, y_test_pred, 'Test')

    print('-'*80, end='\n\n')

Fold 1 of 5
Fitting NuSVR ...
Hold out score of NuSVR: 11.7809
Test score of NuSVR: 12.4200
Fitting KernelRidge ...
Hold out score of KernelRidge: 11.5135
Test score of KernelRidge: 12.3188
Fitting MLPRegressor ...
Hold out score of MLPRegressor: 11.4951
Test score of MLPRegressor: 12.3116
Fitting LGBMRegressor ...
Hold out score of LGBMRegressor: 11.3933
Test score of LGBMRegressor: 12.2188
Fitting XGBRegressor ...
Hold out score of XGBRegressor: 11.3665
Test score of XGBRegressor: 12.1933
Fitting CatBoostRegressor ...
Hold out score of CatBoostRegressor: 11.3001
Test score of CatBoostRegressor: 12.1977
Meta regressor: LinearRegression, RMSE hold out: 11.316235208791605, RMSE test: 12.1932, fit time: 128.73 s, Coefficients: [ 0.01579481  0.17068665 -0.03348082  0.21198083 -0.01035021  0.65979126], intercept: -0.410370317095591, l1_ratio: nan, alpha: nan
Meta regressor: RidgeCV, RMSE hold out: 11.322984032132483, RMSE test: 12.1878, fit time: 129.00 s, Coefficients: [ 0.02074488  0.171

In [6]:
# For final submission, no cv
# CV stacking: 
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

set_config(transform_output="pandas")  # set to pandas
lgbm_params = {'num_leaves': 161, 'max_depth': 2, 'learning_rate': 0.03189823639272201, 'n_estimators': 277, 'subsample': 0.7623316885643929, 'subsample_freq': 8, 
               'colsample_bytree': 0.6539749148962364, 'reg_alpha': 0.10648649504538502, 'random_state': SEED}
xgb_params = {'booster': 'dart', 'max_depth': 2, 'learning_rate': 0.012642475908088945, 'n_estimators': 855, 'subsample': 0.8121633566805094, 
              'colsample_bytree': 0.7117066890547505, 'reg_alpha': 0.005539220298259456, 'reg_lambda': 0.7699973173912055, 'random_state': SEED}
catboost_params = {'learning_rate': 0.037467902179125184, 'n_estimators': 205, 'max_depth': 5, 'l2_leaf_reg': 4.357005411240157, 'rsm': 0.7362139736946606, 
                   'bagging_temperature': 0.8648719006111018, 'silent': True, 'random_state': SEED, 'eval_metric': 'RMSE'}

regressors = {'NuSVR': make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
    FeatureEngineering(level=1),
    PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
    TransformedTargetRegressor(regressor=NuSVR(nu=0.15,
                                               kernel='rbf',
                                               tol=0.001), 
                                               transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
        ),
    'KernelRidge': make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=2),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=KernelRidge(alpha=0.25, kernel='rbf', degree=5, gamma=0.004, coef0=0.5),
                                       transformer=QuantileTransformer(output_distribution='normal', random_state=SEED))
        ),
    'MLPRegressor': make_pipeline(
            PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
            FeatureEngineering(level=1),
            PowerTransformer(method='yeo-johnson', standardize=True, copy=True),
            TransformedTargetRegressor(regressor=MLPRegressor(activation='logistic', max_iter=1000,solver='adam', tol=0.0001)
                                       )
        ),
    'LGBMRegressor': LGBMRegressor(**lgbm_params),
    'XGBRegressor': XGBRegressor(**xgb_params),
    'CatBoostRegressor': CatBoostRegressor(**catboost_params),
}

meta_regressors = [
    # ('LinearRegression', LinearRegression()),
    ('RidgeCV', RidgeCV(alphas=np.logspace(-3, 3, 13), cv=cv)),
    # ('ElasticNetCV', ElasticNetCV(alphas=np.logspace(-3, 3, 13), cv=cv, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=100000)),
    # ('LassoCV', LassoCV(alphas=np.logspace(-3, 3, 13), cv=cv, max_iter=100000)),  # = ElasticNetCV with l1_ratio=1
    # ('LarsCV', LarsCV(cv=cv, max_iter=100000, n_jobs=-1)),
    # ('OrthogonalMatchingPursuitCV', OrthogonalMatchingPursuitCV(cv=cv, n_jobs=-1)),
    # ('LassoLarsCV', LassoLarsCV(cv=cv, max_iter=10000, n_jobs=-1)),
    # ('BayesianRidge', BayesianRidge()),
]

t0 = time.time()

X_meta_train = pd.DataFrame(index=X_train.index, columns=[name for name, _ in regressors.items()])
X_meta_test = pd.DataFrame(index=X_test.index, columns=[name for name, _ in regressors.items()])

# for name, regressor in regressors:
for name, regressor in regressors.items():
    print(f'Fitting {name} ...')
    
    if name == 'CatBoostRegressor':
        X_meta_train[name] = cross_val_predict(regressor, X_train, y_train, cv=cv, verbose=0)  # CatBoostRegressor fails on n_jobs=-1
    else:
        X_meta_train[name] = cross_val_predict(regressor, X_train, y_train, cv=cv, n_jobs=-1, verbose=0)
    
    # fit the model on the full cv training set
    regressor.fit(X_train, y_train)
    X_meta_test[name] = regressor.predict(X_test)


for name, meta_regressor in meta_regressors:
    meta_regressor.fit(X_meta_train, y_train)
    y_test_pred = meta_regressor.predict(X_meta_test)
    
    l1_ratio = getattr(meta_regressor, 'l1_ratio_', np.nan)
    alpha = getattr(meta_regressor, 'alpha_', np.nan)
    coef = getattr(meta_regressor, 'coef_', np.nan)
    intercept = getattr(meta_regressor, 'intercept_', np.nan)
    
    print(f'Meta regressor: {name}, fit time: {time.time() - t0:.2f} s,',
            f'Coefficients: {coef}, intercept: {intercept}, l1_ratio: {l1_ratio}, alpha: {alpha}', end='\n'
    )

print('-'*80, end='\n\n')

Fold 3 of 5
Fitting NuSVR ...
Fitting KernelRidge ...
Fitting MLPRegressor ...
Fitting LGBMRegressor ...
Fitting XGBRegressor ...
Fitting CatBoostRegressor ...


AttributeError: 'list' object has no attribute 'fit'

In [17]:
meta_regressors = [
    # ('LinearRegression', LinearRegression()),
    # ('RidgeCV', RidgeCV(alphas=np.logspace(-3, 3, 13), cv=cv)),
    ('ElasticNetCV', ElasticNetCV(alphas=np.logspace(-3, 3, 13), cv=cv, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=100000)),
    # ('LassoCV', LassoCV(alphas=np.logspace(-3, 3, 13), cv=cv, max_iter=100000)),  # = ElasticNetCV with l1_ratio=1
    # ('LarsCV', LarsCV(cv=cv, max_iter=100000, n_jobs=-1)),
    # ('OrthogonalMatchingPursuitCV', OrthogonalMatchingPursuitCV(cv=cv, n_jobs=-1)),
    # ('LassoLarsCV', LassoLarsCV(cv=cv, max_iter=10000, n_jobs=-1)),
    # ('BayesianRidge', BayesianRidge()),
]

for name, meta_regressor in meta_regressors:
    meta_regressor.fit(X_meta_train, y_train)
    y_test_pred = meta_regressor.predict(X_meta_test)
    
    l1_ratio = getattr(meta_regressor, 'l1_ratio_', np.nan)
    alpha = getattr(meta_regressor, 'alpha_', np.nan)
    coef = getattr(meta_regressor, 'coef_', np.nan)
    intercept = getattr(meta_regressor, 'intercept_', np.nan)
    
    print(f'Meta regressor: {name}, fit time: {time.time() - t0:.2f} s,',
            f'Coefficients: {coef}, intercept: {intercept}, l1_ratio: {l1_ratio}, alpha: {alpha}', end='\n'
    )

print('-'*80, end='\n\n')

Meta regressor: ElasticNetCV, fit time: 592.20 s, Coefficients: [ 0.00565402  0.08542782  0.14529073 -0.25168659  0.28579873  0.74448237], intercept: -0.456727903601454, l1_ratio: 0.1, alpha: 0.1
--------------------------------------------------------------------------------



# Submission

In [18]:
# Save predictions
sub = pd.read_csv('submissions/sample_submission.csv')
sub[TARGET] = y_test_pred
now = time.strftime("%Y-%m-%d %H_%M_%S")
sub.to_csv(f'submissions/submission_StackingElasticNetCV_{now}.csv', index=False)